# Imports (No Changes Required)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cv2 as cv
import typing as t
import pandas as pd
import pathlib as pb
import numpy as np
import sklearn as sk
import random as rnd
import matplotlib.pyplot as plt
import torch
import torchvision as tv
import torch.utils.data as data
from torch import Tensor
import lightning.pytorch as pl

In [3]:
import domino.data
import domino.game
import domino.model
import domino.vision
import domino.preprocess
from domino.model import CellClassifier

# Environment Setup  (No Changes Required)

In [4]:
# Predefined expected paths
ROOT_PATH: pb.Path = pb.Path('.')
HELP_PATH: pb.Path = ROOT_PATH / 'template'
CACHE_DIR: pb.Path = ROOT_PATH / '.cache'
MODEL_DIR: pb.Path = ROOT_PATH / '.model'
SKLRN_DIR: pb.Path = MODEL_DIR / 'sklearn'
TORCH_DIR: pb.Path = MODEL_DIR / 'torch'

# Pretrained model weights
FOREST_MODEL_PATH: pb.Path = SKLRN_DIR / 'balanced-random-forest-full.joblib'
RESNET_MODEL_PATH: pb.Path = TORCH_DIR / 'resnet50-finetune-full.ckpt'
SVM_MODEL_PATH: pb.Path    = SKLRN_DIR / 'svm-full.joblib'

# Obtain deterministic results
SEED = 42
rnd.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
generator: torch.Generator = torch.Generator().manual_seed(SEED)

# Configure current environment
torch.hub.set_dir(CACHE_DIR)
torch.set_float32_matmul_precision('medium')

# Adjust Input and Output Paths (Changes Required!)

In [5]:
# Modify HERE: 
TEST_PATH_INPUT : pb.Path = ROOT_PATH / '..' / 'data' / 'valid'  / 'input'
TEST_PATH_OUTPUT: pb.Path = ROOT_PATH / '..' / 'data' / 'valid'  / 'output'

# Run the Task Solver

In [6]:
# Construct the possible classifiers
classifiers: t.List[CellClassifier] = [
    CellClassifier.load_checkpoint(RESNET_MODEL_PATH, 'resnet'),
    CellClassifier.load_checkpoint(SVM_MODEL_PATH, 'svm'),
    CellClassifier.load_checkpoint(FOREST_MODEL_PATH, 'forest'),
]

# Construct task solver
task_solver = domino.game.DoubleDoubleDominoesTask(
    dataset_path=TEST_PATH_INPUT,
    template_path=HELP_PATH,
    output_path=TEST_PATH_OUTPUT,
    classifiers=classifiers,
    cell_splitter='hough',
    show_matrix=False,
    show_image=False,
    train=False,
    debug=False,
)

# Run the task solver
task_solver.solve()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/invokariman/.cache/pypoetry/virtualenvs/double-double-dominos-R-JT3pI_-py3.10/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 29/29 [00:00<00:00, 54.80it/s]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 29/29 [00:00<00:00, 197.59it/s]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Predicting DataLoader 0: 100%|██████████| 29/29 [00:00<00:00, 206.59it/s]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 29/29 [00:00<00:00, 202.53it/s]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 29/29 [00:00<00:00, 208.23it/s]
